# Glassdoor - Companies and Ratings

In [1]:
import  requests
import urllib.request
import csv
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
import time

In [2]:
#Employer.csv contains all the employers retrieved via web scraping job posts. 
#Here will be using those values for the 'employer' parameter while making request via Glassdoor API

employer_df = pd.read_csv("Employer.csv")
print(employer_df.shape[0])

1579


In [3]:
employer_df[417:418]

,Employer
417,Docusign


### Getting ratings for the companies in the Employer.csv

We are using public Glassdoor API id (t.p=25738) and key (t.k=iRCtcWJQamE) provided via blog post: [Get Glassdoor Data from their API using Python](https://oonlab.com/get-glassdoor-data-from-their-api-using-python). This API key was gained 3 years ago when Glassdoor had free API sign up which they don’t provide any more until further notice [(link)](https://help.glassdoor.com/article/Glassdoor-API/en_US). Also, this API key has come with following restrictions on Glassdoor API Usage: [Public API Terms of Use](https://www.glassdoor.com/crs/api/glassdoor-public-api-terms.pdf). We have decided to use this public API id and key until we get our partnership inquiry permitted.

Note: We have submitted [partnership inquiry](https://help.glassdoor.com/ContactUs/en_US) several times during the fits month of Final Team Software Project, but with no respond from Glassdoor.

Glassdoor API request, used in the blog post, retrieves data about companies ratings. By reading [Glassdoor API Overview](https://www.glassdoor.com/developer/index.htm) and guidance in regards to parameters, we come to conclusion that by changing value of parameter ‘action’ from ‘action = employer’ to:
‘action = salaries’ we will fetch data about salaries, and
‘action = interviews’ we will fetch data about interview difficulty.

#### Job Postings Data
We weren’t able to find a value of parameter ‘action’ that would retrieve data about job postings, hence we decided to use web scraping for that quest. Tech topics from Meetup were used to retrieve corresponding job posts for Dublin. 

In [4]:
URLs=[]
for i in range(employer_df.shape[0]):
    parametars={'t.p':25738,'t.k':'iRCtcWJQamE','userip':'0.0.0.0','format':'json','v':1,'action':'employers','employer':employer_df['Employer'][i]}
    r = requests.get("http://api.glassdoor.com/api/api.htm", params=parametars)
    print(r.url)
    URLs.append(r.url)
    
len(URLs)

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=2+Into+3
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=2+Sister+Food+Group
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=2Source+Talent
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=3+Ireland
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=3D+Design+Bureau
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=3D+Personnel
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=8+West+Consulting
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJ

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Allianz+Worldwide+Partners
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Allied+Irish+Bank
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Allied+Irish+Banks
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Allied+Worldwide
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Allnone
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Allsaints+Retail
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Alltrans
http://api.glassdoor.com/api/api.htm?t.p=2573

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Avery+Dennison
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Aviva
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Aviva+Canada
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Axa+Insurance
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Axios+International+Consultants
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Axonista
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Axway
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&us

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Bny+Mellon
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Boc+Group
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Boeing
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Boi+Payment+Acceptance
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Books-A-Million
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Boots
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Bord+Bia
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Cargurus
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Carl+Zeiss
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Carlson+Wagonlit+Travel
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Carmichael+Uk
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Carna+Transport+Ireland
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Carr+Golf
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Carraig+Donn
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCt

...

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Tjx+Companies
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Tk+Maxx
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Tmf+Group
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Toast
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Tommy+Hilfiger
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Top+Recruitment
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Top+Tier+Recruitment
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Vaping+Wholesale+Uk
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Varonis
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Vectra+Networks
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Venn+Life+Sciences
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Veolia+Environmental+Services
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Veritas
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Verizon
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Wrike
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Www.Jobsireland.Ie
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=X4+Group
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=X4+Rec
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Xanadu
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Xanadu+Consultancy
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Xerox
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=js

http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Journal+Media
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=KBC+Private+Equity
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Keywords+International
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Liberty+IT
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Maynooth+University
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Monarch+Recruitment
http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=MonetizeMore
http://api.glassdoor.com/api/api.htm?t.p

1579

In [5]:
URLs[417]

'http://api.glassdoor.com/api/api.htm?t.p=25738&t.k=iRCtcWJQamE&userip=0.0.0.0&format=json&v=1&action=employers&employer=Docusign'

In [47]:
file = open('URLs_employers','w');
file.write(str(URLs));
file.close();

In [6]:
employer_df[1246:1247]

,Employer
1246,Sodexo


In [25]:
hdr = {'User-Agent': 'Mozilla/5.0'}
ratings_list_nested = []
for i in range(len(URLs)):
    print(i)
    url = URLs[i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep(0.1)
    soup = BeautifulSoup(response, 'lxml')
    ratings_data=json.loads(soup.p.get_text())
    ratings_list_nested.append(pd.DataFrame(ratings_data['response']['employers']))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229


HTTPError: HTTP Error 403: Forbidden

In [32]:
hdr = {'User-Agent': 'Mozilla/5.0'}
for i in range(len(URLs)-1274):
    print(1274+i)
    url = URLs[1274+i]
    req = urllib.request.Request(url,headers=hdr)
    response = urllib.request.urlopen(req)
    time.sleep(0.3)
    soup = BeautifulSoup(response, 'lxml')
    ratings_data=json.loads(soup.p.get_text())
    ratings_list_nested.append(pd.DataFrame(ratings_data['response']['employers']))

1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409
1410
1411
1412
1413
1414
1415
1416
1417
1418
1419
1420
1421
1422
1423
1424
1425
1426
1427
1428
1429
1430
1431
1432
1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473


In [37]:
ratings_list_nested[200]

,careerOpportunitiesRating,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,id,industry,isEEP,name,numberOfRatings,overallRating,ratingDescription,recommendToFriendRating,seniorLeadershipRating,squareLogo,website,workLifeBalanceRating
0,0.0,0.0,0.0,True,3055137,,False,Breagh Recruitment,0,0,Not Applicable,0,0.0,,www.breaghrecruitment.com,0.0


In [38]:
len(ratings_list_nested)

1577

In [39]:
company_rating_df_nested = pd.DataFrame()
for i in range(len(ratings_list_nested)):
    company_rating_df_nested=company_rating_df_nested.append(ratings_list_nested[i])
company_rating_df_nested

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,id,industry,industryId,industryName,...,overallRating,parentEmployer,ratingDescription,recommendToFriendRating,sectorId,sectorName,seniorLeadershipRating,squareLogo,website,workLifeBalanceRating
0,2.3,"{'name': 'Ranjit Singh', 'title': 'CEO', 'numb...",2.1,2.3,False,{'attributionURL': 'https://www.glassdoor.com/...,523766,Food & Beverage Manufacturing,200071.0,Food & Beverage Manufacturing,...,2.7,NaN,OK,29,10015.0,Manufacturing,2.3,https://media.glassdoor.com/sqll/523766/2-sist...,www.2sfg.com,2.8
0,5.0,NaN,4.4,5.0,True,NaN,1839855,IT Services,200064.0,IT Services,...,4.8,NaN,Very Satisfied,100,10013.0,Information Technology,5.0,https://media.glassdoor.com/sqll/1839855/2sour...,www.2sourcetalent.com,4.4
0,2.9,"{'name': 'Robert Finnegan', 'title': 'CEO', 'n...",3.4,3.2,False,{'attributionURL': 'https://www.glassdoor.com/...,39331,Telecommunications Services,200122.0,Telecommunications Services,...,3.4,NaN,OK,50,10023.0,Telecommunications,2.9,https://media.glassdoor.com/sqll/39331/3-irela...,www.three.ie,3.5
0,3.9,"{'name': 'Nicholas Polley', 'title': 'Managing...",3.9,3.9,True,NaN,2880108,Architectural & Engineering Services,200023.0,Architectural & Engineering Services,...,3.9,NaN,Satisfied,72,10006.0,Business Services,3.9,https://media.glassdoor.com/sqll/2880108/3d-de...,www.3ddesignbureau.com,3.9
0,2.7,"{'name': 'Damian Doherty', 'title': 'Owner', '...",2.8,2.8,True,NaN,1193024,Staffing & Outsourcing,200032.0,Staffing & Outsourcing,...,3.2,NaN,OK,58,10006.0,Business Services,3.3,https://media.glassdoor.com/sqll/1193024/3d-pe...,www.3dpersonnel.com,2.9
0,5.0,NaN,5.0,5.0,False,{'attributionURL': 'https://www.glassdoor.com/...,1169343,,NaN,NaN,...,5.0,NaN,Very Satisfied,100,NaN,NaN,5.0,,www.aattech.com,5.0
0,3.5,"{'name': 'Peter Voser', 'title': 'Interim Pres...",3.5,3.7,True,{'attributionURL': 'https://www.glassdoor.com/...,3076,Electrical & Electronic Manufacturing,200070.0,Electrical & Electronic Manufacturing,...,3.7,NaN,Satisfied,77,10015.0,Manufacturing,3.2,https://media.glassdoor.com/sqll/3076/abb-squa...,www.abb.com,3.7
1,2.8,"{'name': 'Peter Voser', 'title': 'ABB Global C...",3.5,3.0,False,{'attributionURL': 'https://www.glassdoor.com/...,73,Electrical & Electronic Manufacturing,200070.0,Electrical & Electronic Manufacturing,...,3.0,"{'id': 3076, 'name': 'ABB', 'relationshipDate'...",OK,53,10015.0,Manufacturing,2.8,https://media.glassdoor.com/sqll/73/baldor-ele...,baldor.abb.com,3.1
2,2.9,"{'name': 'Thomas Burke', 'title': 'CEO', 'numb...",3.2,3.1,False,{'attributionURL': 'https://www.glassdoor.com/...,485895,Wholesale,200033.0,Wholesale,...,3.1,NaN,OK,48,10006.0,Business Services,2.9,https://media.glassdoor.com/sqll/485895/abb-co...,www.abboptical.com,3.2
3,3.3,NaN,3.9,3.9,False,{'attributionURL': 'https://www.glassdoor.com/...,748075,Consulting,200028.0,Consulting,...,3.9,NaN,Satisfied,100,10006.0,Business Services,3.5,,www.abb-consultant.com,3.6


In [42]:
company_rating_df_nested["name"]

0                          2 Sisters Food Group
0                                2Source Talent
0                                 Three Ireland
0                              3D Design Bureau
0                                  3D Personnel
0                                AAT Technology
0                                           ABB
1               ABB Motors and Mechanical, Inc.
2                             ABB Optical Group
3                                ABB Consultant
4                                    Sobetronic
5     ABB Tropos Wireless Communication Systems
6                                 ABB no Brasil
7                                 Abb do Brasil
8                                     ABB Group
9                              ABB InformÃ¡tica
0                                        Abbott
1                          Abbott & Abbott, P.C
2                           Abbott Laboratories
3                               Abbott Vascular
4                              Abbott Nu

In [45]:
company_rating_df_nested.to_csv(r'/Users/brianyang/Documents/DS_GA1007/project/Data_Glassdoor.csv')

In [13]:
#Adding new columns because of the nested featuredReview column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list=['review_URL','review_ID','review_currentJob','review_DateTime',
              'review_jobTitle','review_location','review_pros','review_cons','review_overall']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list])

In [14]:
split_featuredReview = company_rating_df_nested['featuredReview'].apply(pd.Series) # split the featuredReview column from company_rating_df_nested
split_featuredReview = split_featuredReview.reset_index(drop=True) # reset index

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['review_URL'].iloc[i]=split_featuredReview['attributionURL'][i]
    company_rating_df_nested['review_ID'].iloc[i]=split_featuredReview['id'][i]
    company_rating_df_nested['review_currentJob'].iloc[i]=split_featuredReview['currentJob'][i]
    company_rating_df_nested['review_DateTime'].iloc[i]=split_featuredReview['reviewDateTime'][i]
    company_rating_df_nested['review_jobTitle'].iloc[i]=split_featuredReview['jobTitle'][i]
    company_rating_df_nested['review_location'].iloc[i]=split_featuredReview['location'][i]
    company_rating_df_nested['review_pros'].iloc[i]=split_featuredReview['pros'][i]
    company_rating_df_nested['review_cons'].iloc[i]=split_featuredReview['cons'][i]
    company_rating_df_nested['review_overall'].iloc[i]=split_featuredReview['overall'][i]


/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
#Adding new columns because of the nested ceo column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list2=['ceo_name','ceo_title','ceo_numberOfRatings','ceo_approval_rate','ceo_Photo_URL']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list2])

In [16]:
split_ceo = company_rating_df_nested['ceo'].apply(pd.Series) # split the ceo column from company_rating_df_nested
split_ceo = split_ceo.reset_index(drop=True) # reset index
split_ceo_2 = split_ceo['image'].apply(pd.Series)

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['ceo_name'].iloc[i]=split_ceo['name'][i]
    company_rating_df_nested['ceo_title'].iloc[i]=split_ceo['title'][i]
    company_rating_df_nested['ceo_numberOfRatings'].iloc[i]=split_ceo['numberOfRatings'][i]
    company_rating_df_nested['ceo_approval_rate'].iloc[i]=split_ceo['pctApprove'][i]
    company_rating_df_nested['ceo_Photo_URL'].iloc[i]=split_ceo_2['src'][i]
#company_rating_df_nested.head()

/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [35]:
#Adding new columns because of the nested parentEmployer column
#df.reindex(columns=[*df.columns.tolist(), 'new_column1', 'new_column2'], fill_value=0)

my_cols_list3=['parentEmployer_name','parentEmployer_ID','parentEmployer_logoURL']
company_rating_df_nested=company_rating_df_nested.reindex(columns=[*company_rating_df_nested.columns.tolist(), *my_cols_list3])

In [36]:
split_parentEmployer = company_rating_df_nested['parentEmployer'].apply(pd.Series) # split the parentEmployer column from company_rating_df_nested
split_parentEmployer = split_parentEmployer.reset_index(drop=True) # reset index
split_parentEmployer_2 = split_parentEmployer['logo'].apply(pd.Series)

for i in range(len(company_rating_df_nested)):
    company_rating_df_nested['parentEmployer_name'].iloc[i]=split_parentEmployer['name'][i]
    company_rating_df_nested['parentEmployer_ID'].iloc[i]=split_parentEmployer['id'][i]
    company_rating_df_nested['parentEmployer_logoURL'].iloc[i]=split_parentEmployer_2['normalUrl'][i]


#company_rating_df_nested.head()

/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/aidasehic/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [45]:
company_rating_df_nested

,careerOpportunitiesRating,ceo,compensationAndBenefitsRating,cultureAndValuesRating,exactMatch,featuredReview,company_id,industryId,industryName,company_name,...,review_cons,review_overall,ceo_name,ceo_title,ceo_numberOfRatings,ceo_approval_rate,ceo_Photo_URL,parentEmployer_name,parentEmployer_ID,parentEmployer_logoURL
0,2.7,"{'name': 'Ranjit Singh', 'title': 'CEO', 'numb...",2.2,2.5,False,{'attributionURL': 'https://www.glassdoor.com/...,523766,200071.0,Food & Beverage Manufacturing,2 Sisters Food Group,...,"Night shifts, noisy and due to quality require...",4.0,Ranjit Singh,CEO,24.0,65.0,https://media.glassdoor.com/people/sqll/523766...,NaN,NaN,NaN
0,5.0,NaN,4.4,5.0,True,NaN,1839855,200064.0,IT Services,2Source Talent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3.0,"{'name': 'Robert Finnegan', 'title': 'CEO', 'n...",3.4,3.1,False,{'attributionURL': 'https://www.glassdoor.com/...,39331,200122.0,Telecommunications Services,Three Ireland,...,"Car park is usually packed, if you are first i...",5.0,Robert Finnegan,CEO,38.0,63.0,https://media.glassdoor.com/people/sqll/39331/...,NaN,NaN,NaN
0,2.1,"{'name': 'Damian Doherty', 'title': 'Owner', '...",2.3,2.3,True,{'attributionURL': 'https://www.glassdoor.com/...,1193024,200032.0,Staffing & Outsourcing,3D Personnel,...,There can be days you are available for work a...,3.0,Damian Doherty,Owner,3.0,68.0,NaN,NaN,NaN,NaN
0,5.0,NaN,5.0,5.0,False,NaN,1169343,NaN,NaN,AAT Technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3.3,"{'name': 'Ulrich Spiesshofer', 'title': 'Chief...",3.3,3.6,True,{'attributionURL': 'https://www.glassdoor.com/...,3076,200070.0,Electrical & Electronic Manufacturing,ABB,...,Low salary and slow paced,4.0,Ulrich Spiesshofer,Chief Executive Officer,731.0,83.0,https://media.glassdoor.com/people/sqll/3076/a...,NaN,NaN,NaN
1,3.0,"{'name': 'Angel Alvarez', 'title': 'Chief Exec...",3.3,3.0,False,NaN,485895,200033.0,Wholesale,ABB Optical Group,...,NaN,NaN,Angel Alvarez,Chief Executive Officer,51.0,77.0,NaN,NaN,NaN,NaN
2,3.3,NaN,3.9,3.9,False,NaN,748075,200028.0,Consulting,ABB Consultant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,5.0,5.0,False,NaN,908157,200059.0,Health Care Services & Hospitals,ABB Tropos Wireless Communication Systems,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.3,"{'name': 'Matthias Heilmann', 'title': 'Produc...",3.1,2.4,False,NaN,3233,200060.0,Computer Hardware & Software,Ventyx,...,NaN,NaN,Matthias Heilmann,Product Group Manager,16.0,12.0,NaN,ABB,3076.0,https://media.glassdoor.com/sql/3076/abb-squar...


In [51]:
#dropping columns'ceo','featuredReview', 'parentEmployer'
company_rating_df2=company_rating_df_nested.drop(['ceo','featuredReview', 'parentEmployer'], axis=1)

In [52]:
#Removing duplicate rows
company_rating_df_no_duplicates = company_rating_df2.drop_duplicates(subset=None, keep='first', inplace=False)

In [54]:
#Changing order of the columns
company_rating_df = company_rating_df_no_duplicates[['company_name', 'company_id', 'website', 'company_logo_url', 
                                              'parentEmployer_name', 'parentEmployer_ID','parentEmployer_logoURL',
                                              'numberOfRatings', 'overallRating', 'ratingDescription',
                                              'workLifeBalanceRating', 'careerOpportunitiesRating', 
                                              'compensationAndBenefitsRating', 'cultureAndValuesRating',
                                              'recommendToFriendRating', 'seniorLeadershipRating', 'ceo_name', 
                                              'ceo_title', 'ceo_Photo_URL', 'ceo_numberOfRatings', 'ceo_approval_rate',
                                              'review_URL', 'review_ID', 'review_pros', 'review_cons', 'review_overall',
                                              'review_location', 'review_DateTime', 'review_currentJob', 
                                              'review_jobTitle', 'review_location', 'industryName', 'industryId',
                                              'sectorName', 'sectorId', 'exactMatch']]

In [56]:
company_rating_df.to_csv('company_overall_ratings_all.csv', sep = ',', index = False) # create csv for salaries
company_rating_df

,company_name,company_id,website,company_logo_url,parentEmployer_name,parentEmployer_ID,parentEmployer_logoURL,numberOfRatings,overallRating,ratingDescription,...,review_location,review_DateTime,review_currentJob,review_jobTitle,review_location,industryName,industryId,sectorName,sectorId,exactMatch
0,2 Sisters Food Group,523766,www.2sfg.com,https://media.glassdoor.com/sqll/523766/2-sist...,NaN,NaN,NaN,54,3.0,OK,...,"Longford, Co. Longford (Ireland)",2015-11-24 00:29:59.22,False,Employee,"Longford, Co. Longford (Ireland)",Food & Beverage Manufacturing,200071.0,Manufacturing,10015.0,False
0,2Source Talent,1839855,www.2sourcetalent.com,https://media.glassdoor.com/sqll/1839855/2sour...,NaN,NaN,NaN,5,4.8,Very Satisfied,...,NaN,NaN,NaN,NaN,NaN,IT Services,200064.0,Information Technology,10013.0,True
0,Three Ireland,39331,www.three.ie,https://media.glassdoor.com/sqll/39331/3-irela...,NaN,NaN,NaN,65,3.2,OK,...,"Dublin, Co. Dublin (Ireland)",2018-05-22 01:07:23.073,True,Employee,"Dublin, Co. Dublin (Ireland)",Telecommunications Services,200122.0,Telecommunications,10023.0,False
0,3D Personnel,1193024,www.3dpersonnel.com,https://media.glassdoor.com/sqll/1193024/3d-pe...,NaN,NaN,NaN,6,2.8,OK,...,"Dublin, Co. Dublin (Ireland)",2017-11-14 02:21:03.443,True,General Operator,"Dublin, Co. Dublin (Ireland)",Staffing & Outsourcing,200032.0,Business Services,10006.0,True
0,AAT Technology,1169343,www.aattech.com,,NaN,NaN,NaN,1,5.0,Very Satisfied,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
0,ABB,3076,www.abb.com,https://media.glassdoor.com/sqll/3076/abb-squa...,NaN,NaN,NaN,2426,3.6,Satisfied,...,,2017-06-30 08:22:17.303,False,Employee,,Electrical & Electronic Manufacturing,200070.0,Manufacturing,10015.0,True
1,ABB Optical Group,485895,www.abboptical.com,https://media.glassdoor.com/sqll/485895/abb-co...,NaN,NaN,NaN,85,3.1,OK,...,NaN,NaN,NaN,NaN,NaN,Wholesale,200033.0,Business Services,10006.0,False
2,ABB Consultant,748075,www.abb-consultant.com,,NaN,NaN,NaN,4,3.9,Satisfied,...,NaN,NaN,NaN,NaN,NaN,Consulting,200028.0,Business Services,10006.0,False
3,ABB Tropos Wireless Communication Systems,908157,www.tropos.com,,NaN,NaN,NaN,1,5.0,Very Satisfied,...,NaN,NaN,NaN,NaN,NaN,Health Care Services & Hospitals,200059.0,Health Care,10012.0,False
4,Ventyx,3233,www.abb.com/enterprise-software,https://media.glassdoor.com/sqll/3233/ventyx-s...,ABB,3076.0,https://media.glassdoor.com/sql/3076/abb-squar...,194,2.3,Dissatisfied,...,NaN,NaN,NaN,NaN,NaN,Computer Hardware & Software,200060.0,Information Technology,10013.0,False
